In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from iss_analysis.io import get_mcherry_cells

project = 'becalia_rabies_barseq'
mouse = 'BRAC8498.3e'

# Get the good mCherry cells, i.e. the one for which we have masks and we match the 
# starters
mcherry_curated = get_mcherry_cells(
    project, mouse, verbose=True, which="curated", prefix="mCherry_1"
)
# For chamber6, detection fails because of high background, use the manual click instead
mcherry_manual =  get_mcherry_cells(
    project, mouse, verbose=True, which="manual", prefix="mCherry_1"
)

m6 = mcherry_manual.query("chamber == 'chamber_06'")
n6 = len(m6)
total = len(mcherry_curated) + n6

print(f"{n6} cells in chamber 6 out of {total}, that's {n6/total*100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt

# Plot manual for chamber 6
m6 = mcherry_manual.query("chamber == 'chamber_06'")
m6 = m6.groupby('roi').aggregate(len).x
plt.plot(m6.index.astype(int)-10, m6.values, 'o-')

# Plot curated for the other
n=10
for c in ['07','08','09','10']:
    df = mcherry_curated.query(f"chamber == 'chamber_{c}'")
    d = df.groupby('roi').aggregate(len).x
    plt.plot(d.index + n, d.values, 'o-')
    n += d.index.max()

plt.xlabel('Slice index')
plt.ylabel('# of mcherry cells')

In [ ]:
# Add the starter position
from iss_analysis.io import get_starter_cells, get_sections_info
import pandas as pd
import numpy as np

sec_inf = get_sections_info(project, mouse, chamber=None)
starters_positions = get_starter_cells(project, mouse)

fig, ax = plt.subplots(1,1, figsize=(4,3))
axt = ax.twinx()
ax.plot((m6.index.astype(int)-10)*20, m6.values, 'o-', color='grey')
n = 10

for c in ['07','08','09','10']:
    df = mcherry_curated.query(f"chamber == 'chamber_{c}'")
    d = df.groupby('roi').aggregate(len).x
    ax.plot((d.index + n)*20, d.values, 'o-', color='darkred', label='mCherry cells' if c=='10' else '__no_label__')
    sdf = starters_positions.query(f"chamber == 'chamber_{c}'")
    if len(sdf):
        sd = sdf.groupby('roi').aggregate(len).y
        axt.plot((sd.index + n)*20, sd.values, 's-', color='dodgerblue', label='Starter cells' if c=='10' else '__no_label__') 
    n += d.index.max()
axt.set_ylabel('# of starter cells', color='dodgerblue')
ax.set_yticks(np.arange(0,201,50))
ax.set_yticklabels(np.arange(0,201,50), color='darkred')
ax.set_ylim(0, 200)
axt.set_yticks(np.arange(0, 81, 20))
axt.set_yticklabels(np.arange(0, 81, 20), color='dodgerblue')
axt.set_ylim(0, 80)
ax.set_xlabel(r'Slice A/P position ($\mu m$)')
ax.set_ylabel('# of mCherry cells', color='darkred')
fig.tight_layout()